In [11]:
import requests
from bs4  import BeautifulSoup
import pandas as pd
import numpy as np

from time import sleep
from random import randint
from warnings import warn
from time import time

## Restaurant

In [12]:
#  1 webscraping Hotels
# Monitoring of the loop
start_time = time()
request = 0

# Dictionary for storing web content
soup_r = {}
for start_num in range(0, 240, 10):

    # Make a request
    url = f" https://www.yelp.com/search?find_desc=Restaurants&find_loc=london&start={start_num}"
    response = requests.get(url)

    # Pause the loop
    sleep(randint(8,15))
    print(response)
    page = BeautifulSoup(response.content, 'html.parser')
    soup_r[start_num] = page


    # Monitor the requests
    request += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(request, request/elapsed_time))
    #clear_output(wait = True)

    # Throw a warning if request wasn't successful
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(request, response.status_code))

<Response [200]>
Request:1; Frequency: 0.09596472414875633 requests/s
<Response [200]>
Request:2; Frequency: 0.08296823969990778 requests/s
<Response [200]>
Request:3; Frequency: 0.07117124583277931 requests/s
<Response [200]>
Request:4; Frequency: 0.07079988726686308 requests/s
<Response [200]>
Request:5; Frequency: 0.06934838338363461 requests/s
<Response [200]>
Request:6; Frequency: 0.07144294816420729 requests/s
<Response [200]>
Request:7; Frequency: 0.06928892643648463 requests/s
<Response [200]>
Request:8; Frequency: 0.07176036082956322 requests/s
<Response [200]>
Request:9; Frequency: 0.0704357032984562 requests/s
<Response [200]>
Request:10; Frequency: 0.07031326682153576 requests/s
<Response [200]>
Request:11; Frequency: 0.07057766788498372 requests/s
<Response [200]>
Request:12; Frequency: 0.07046893431707134 requests/s
<Response [200]>
Request:13; Frequency: 0.07035984746328822 requests/s
<Response [200]>
Request:14; Frequency: 0.07018556494806649 requests/s
<Response [200]>

In [14]:
resto_soup = pd.DataFrame([soup_r]).squeeze()

### Name

In [15]:
name = []
for row in resto_soup:  
    result = row.select('.css-1egxyvc .css-1m051bw') 
    for tag in result:
        #print(tag.text)
        name.append(tag.text)

In [16]:
# pd.Series(name).value_counts()

### Ratings

In [17]:
rating = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in resto_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        rat = header_text.select_one('.overflow--hidden__09f24___ayzG')
        if rat is not None:
            rating.append(rat['aria-label'].split()[0])
        else:
            rating.append(np.nan)

In [18]:
# test
# for n, rat in zip(name, rating):
#     print(n, rat) 

### Reviews

In [19]:
reviews = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in resto_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        rev = header_text.select_one('.reviewCount__09f24__tnBk4')
        if rev is not None:
            reviews.append(rev.text)
        else:
            reviews.append(np.nan)

In [20]:
# test
# for n, rev in zip(name, reviews):
#     print(n, rev) 

### Price

In [41]:
price_range = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in resto_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        price = header_text.select_one('.css-1s7bx9e')
        if price is not None:
            price_range.append(price.text)
        else:
            price_range.append(np.nan)

In [23]:
# test
# for n, price in zip(name, price_range):
#     print(n, price)


# Dataframe

In [51]:
restaurant_df = pd.DataFrame({'Name':name, 'Rating':rating, 'Reviews': reviews, 'Price Range in £': price_range})

In [52]:
# store data
restaurant_df.to_csv('restaurant_data.csv', index=False)

In [61]:
#restaurant_df.head()

In [114]:
#restaurant_df.info()

In [34]:
# Missing values
#restaurant_df.isnull().sum().sort_values(ascending=False)

In [32]:
# duplicates values
#restaurant_df[restaurant_df.duplicated()]

### Changing Price value

In [58]:
# Price range (price per person)
# Based on information from yelp and Quora:
# $= under $10. $$=11-30. $$$=31-60. $$$$= over $61
# Converting to pound

# 1 pound = 1.31 USD
Euro = (pd.Series([10, 11, 30, 31, 60, 61]) * 1.31).apply(lambda x: int(x))
#print(Euro)
mapper = {'£':'under 13', '££':'[14 - 39]', '£££':'[40 - 78]', '££££':'over 79'}
restaurant_df['Price Range'] = restaurant_df['Price Range'].map(mapper)

In [59]:
# Top 20 Most reviews

#restaurant_df.sort_values(by='Reviews', ascending=False).head(20)

In [60]:
# Rating and number of reviews
restaurant_df['Rating'] = restaurant_df['Rating'].apply(lambda x: float(x)) 
restaurant_df['Reviews'] = restaurant_df['Reviews'].apply(lambda x: float(x))

#restaurant_df.describe()

# Hotels

In [62]:
#  1 webscraping Hotels
# Monitoring of the loop
start_time = time()
request = 0

# Dictionary for storing web content
soup_r = {}
for start_num in range(0, 240, 10):

    # Make a request
    url = f"https://www.yelp.com/search?find_desc=Hotels&find_loc=london&start={start_num}"
    response = requests.get(url)

    # Pause the loop
    sleep(randint(8,15))
    print(response)
    page = BeautifulSoup(response.content, 'html.parser')
    soup_r[start_num] = page


    # Monitor the requests
    request += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(request, request/elapsed_time))
    #clear_output(wait = True)

    # Throw a warning if request wasn't successful
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(request, response.status_code))

<Response [200]>
Request:1; Frequency: 0.07463168799767089 requests/s
<Response [200]>
Request:2; Frequency: 0.0810789257055489 requests/s
<Response [200]>
Request:3; Frequency: 0.07639726925847587 requests/s
<Response [200]>
Request:4; Frequency: 0.07302651125520489 requests/s
<Response [200]>
Request:5; Frequency: 0.07431678131883682 requests/s
<Response [200]>
Request:6; Frequency: 0.07423896176165527 requests/s
<Response [200]>
Request:7; Frequency: 0.07653986888991388 requests/s
<Response [200]>
Request:8; Frequency: 0.0731284092638863 requests/s
<Response [200]>
Request:9; Frequency: 0.07274786867575989 requests/s
<Response [200]>
Request:10; Frequency: 0.07377396962673882 requests/s
<Response [200]>
Request:11; Frequency: 0.07514750668655451 requests/s
<Response [200]>
Request:12; Frequency: 0.07646583510638981 requests/s
<Response [200]>
Request:13; Frequency: 0.07563615662989248 requests/s
<Response [200]>
Request:14; Frequency: 0.07557877191219448 requests/s
<Response [200]>


C:\Users\ritth\AppData\Local\Temp\ipykernel_50924\1228871983.py:29: UserWarning: Request: 23; Status code: 504
  warn('Request: {}; Status code: {}'.format(request, response.status_code))


<Response [200]>
Request:24; Frequency: 0.0680124365147564 requests/s


In [63]:
hotel_soup = pd.DataFrame([soup_r]).squeeze()

### Name

In [76]:
name = []
for row in hotel_soup:  
    result = row.select('.css-1egxyvc .css-1m051bw') 
    for tag in result:
        #print(tag.text)
        name.append(tag.text)

In [75]:
#pd.Series(name).value_counts()

### Ratings

In [66]:
rating = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in hotel_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        rat = header_text.select_one('.overflow--hidden__09f24___ayzG')
        if rat is not None:
            rating.append(rat['aria-label'].split()[0])
        else:
            rating.append(np.nan)

In [74]:
# test
# for n, rat in zip(name, rating):
#     print(n, rat) 

### Reviews

In [68]:
reviews = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in hotel_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        rev = header_text.select_one('.reviewCount__09f24__tnBk4')
        if rev is not None:
            reviews.append(rev.text)
        else:
            reviews.append(np.nan)

In [73]:
# test
# for n, rev in zip(name, reviews):
#     print(n, rev) 

### Price

In [70]:
price_range = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in hotel_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        price = header_text.select_one('.css-1s7bx9e')
        if price is not None:
            price_range.append(price.text)
        else:
            price_range.append(np.nan)

In [72]:
# test
# for n, price in zip(name, price_range):
#     print(n, price)

# Dataframe

In [77]:
hotel_df = pd.DataFrame({'Name':name, 'Rating':rating, 'Reviews': reviews, 'Price Range in £': price_range})

In [89]:
# store data
hotel_df.to_csv('hotels_data.csv', index=False)

In [81]:
#hotel_df.head()

In [113]:
#hotel_df.info()

### Changing price value

In [84]:
# 1 pound = 1.31 USD
Euro = (pd.Series([10, 11, 30, 31, 60, 61]) * 1.31).apply(lambda x: int(x))
#print(Euro)
mapper = {'£':'under 13', '££':'[14 - 39]', '£££':'[40 - 78]', '££££':'over 79'}
hotel_df['Price Range'] = hotel_df['Price Range'].map(mapper)

In [86]:
# Top 20 Most reviews

#hotel_df.sort_values(by='Reviews', ascending=False).head(20)

# Pubs

In [87]:
#  1 webscraping Hotels
# Monitoring of the loop
start_time = time()
request = 0

# Dictionary for storing web content
soup_r = {}
for start_num in range(0, 240, 10):

    # Make a request
    url = f"https://www.yelp.com/search?find_desc=Pubs&find_loc=london&start={start_num}"
    response = requests.get(url)

    # Pause the loop
    sleep(randint(8,15))
    print(response)
    page = BeautifulSoup(response.content, 'html.parser')
    soup_r[start_num] = page


    # Monitor the requests
    request += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(request, request/elapsed_time))
    #clear_output(wait = True)

    # Throw a warning if request wasn't successful
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(request, response.status_code))

<Response [200]>
Request:1; Frequency: 0.09728534416232013 requests/s
<Response [200]>
Request:2; Frequency: 0.07222438424306961 requests/s
<Response [200]>
Request:3; Frequency: 0.06950750651243158 requests/s
<Response [200]>
Request:4; Frequency: 0.06737870872104207 requests/s
<Response [200]>
Request:5; Frequency: 0.06511078142667917 requests/s
<Response [200]>
Request:6; Frequency: 0.0655845405873091 requests/s
<Response [200]>
Request:7; Frequency: 0.06872124488666487 requests/s
<Response [200]>
Request:8; Frequency: 0.06889124049484453 requests/s
<Response [200]>
Request:9; Frequency: 0.07096912542643673 requests/s
<Response [200]>
Request:10; Frequency: 0.06978563518168855 requests/s
<Response [200]>
Request:11; Frequency: 0.0699970862149353 requests/s
<Response [200]>
Request:12; Frequency: 0.06953901800838702 requests/s
<Response [200]>
Request:13; Frequency: 0.069880794110377 requests/s
<Response [200]>
Request:14; Frequency: 0.07046597346659499 requests/s
<Response [200]>
Re

In [90]:
pub_soup = pd.DataFrame([soup_r]).squeeze()

### Name

In [92]:
name = []
for row in pub_soup:  
    result = row.select('.css-1egxyvc .css-1m051bw') 
    for tag in result:
        #print(tag.text)
        name.append(tag.text)

In [112]:
#pd.Series(name).value_counts()

### Ratings

In [94]:
rating = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in pub_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        rat = header_text.select_one('.overflow--hidden__09f24___ayzG')
        if rat is not None:
            rating.append(rat['aria-label'].split()[0])
        else:
            rating.append(np.nan)

In [96]:
# test
# for n, rat in zip(name, rating):
#     print(n, rat) 

### Reviews

In [97]:
reviews = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in pub_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        rev = header_text.select_one('.reviewCount__09f24__tnBk4')
        if rev is not None:
            reviews.append(rev.text)
        else:
            reviews.append(np.nan)

In [99]:
# test
# for n, rev in zip(name, reviews):
#     print(n, rev) 

### Price

In [100]:
price_range = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in pub_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        price = header_text.select_one('.css-1s7bx9e')
        if price is not None:
            price_range.append(price.text)
        else:
            price_range.append(np.nan)

In [102]:
# test
# for n, price in zip(name, price_range):
#     print(n, price)

# Dataframe

In [103]:
pub_df = pd.DataFrame({'Name':name, 'Rating':rating, 'Reviews': reviews, 'Price Range in £': price_range})

In [104]:
# store data
pub_df.to_csv('pubs_data.csv', index=False)

In [107]:
#pub_df.head()

In [108]:
#pub_df.info()

### Changing price value

In [109]:
# 1 pound = 1.31 USD
Euro = (pd.Series([10, 11, 30, 31, 60, 61]) * 1.31).apply(lambda x: int(x))
#print(Euro)
mapper = {'£':'under 13', '££':'[14 - 39]', '£££':'[40 - 78]', '££££':'over 79'}
pub_df['Price Range'] = pub_df['Price Range'].map(mapper)

In [111]:
# Top 20 Most reviews

#pub_df.sort_values(by='Reviews', ascending=False).head(20)